In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hajarhesham","key":"67e84a21479743949f97c093059149bb"}'}

In [ ]:
%mkdir ~/.kaggle

In [ ]:
%cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                             title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                 5MB  2022-11-13 15:47:17          13905        396  1.0              
thedevastator/analyzing-credit-card-spending-habits-in-india    Credit Card Spending Habits in India           319KB  2022-12-14 07:30:37           1045         43  1.0              
michals22/coffee-dataset                                        Coffee dataset                                  24KB  2022-12-15 20:02:12           3177         75  1.0              
thedevastator/unlock-profits-with-e-commerce-sales-data         E-Commerce Sales Data

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 54% 14.0M/25.7M [00:00<00:00, 143MB/s]
100% 25.7M/25.7M [00:00<00:00, 184MB/s]


In [ ]:
!unzip /content/imdb-dataset-of-50k-movie-reviews.zip

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
import numpy as np
import pandas as pd
import pylab as plt
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import cohen_kappa_score

from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

from imblearn.over_sampling import SMOTE
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from time import time 
import re 
import spacy

import io
import requests
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
url = '/content/IMDB Dataset.csv'
data = pd.read_csv(url)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
data = data.dropna().reset_index(drop=True)

In [ ]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
#subsample the dataset for fast run
N=1000
r=np.random.permutation(data.shape[0])
r=r[:N]
X_train, X_test, y_train, y_test = train_test_split(data['review'][r], data['sentiment'][r], test_size = .5, shuffle = False)

In [ ]:
print('training data',X_train.shape)
print('test data',X_test.shape)
print('training data',y_train.shape)
print('test data',y_test.shape)

training data (500,)
test data (500,)
training data (500,)
test data (500,)


In [ ]:
sentiments = data['sentiment'].value_counts()
print('Sentiments in entire dataset:\n Positive: {}\n Negative: {}'.format(sentiments[1], sentiments[0]))

Sentiments in entire dataset:
 Positive: 25000
 Negative: 25000


In [ ]:
def get_sentiments(d, _d):
    positive = (d=='positive').sum()
    negative = (d=='negative').sum()
    print('Sentiments in {}:\n Positive: {}\n Negative: {}'.format(_d, positive, negative))

In [ ]:
get_sentiments(y_train, 'Train data')
get_sentiments(y_test, 'Test data')

Sentiments in Train data:
 Positive: 253
 Negative: 247
Sentiments in Test data:
 Positive: 250
 Negative: 250


In [ ]:
corpus = []
for i in range (len(data)):
  reviews = data['review'][i]
  reviews.lower()
  reviews.replace('<br /><br />',' ')
  reviews= re.sub('[^a-zA-Z]',' ',reviews)

  li =  [word for word in word_tokenize(reviews) if word not in stopwords.words('english')]
  reviews = ' '.join(li)
  corpus.append(reviews)
   

In [ ]:
from gensim.models import Word2Vec

In [ ]:
words_in_sentences=[]
for i in corpus:
    words_in_sentences.append(i.split())

In [ ]:
words_in_sentences

[['One',
  'reviewers',
  'mentioned',
  'watching',
  'Oz',
  'episode',
  'hooked',
  'They',
  'right',
  'exactly',
  'happened',
  'br',
  'br',
  'The',
  'first',
  'thing',
  'struck',
  'Oz',
  'brutality',
  'unflinching',
  'scenes',
  'violence',
  'set',
  'right',
  'word',
  'GO',
  'Trust',
  'show',
  'faint',
  'hearted',
  'timid',
  'This',
  'show',
  'pulls',
  'punches',
  'regards',
  'drugs',
  'sex',
  'violence',
  'Its',
  'hardcore',
  'classic',
  'use',
  'word',
  'br',
  'br',
  'It',
  'called',
  'OZ',
  'nickname',
  'given',
  'Oswald',
  'Maximum',
  'Security',
  'State',
  'Penitentary',
  'It',
  'focuses',
  'mainly',
  'Emerald',
  'City',
  'experimental',
  'section',
  'prison',
  'cells',
  'glass',
  'fronts',
  'face',
  'inwards',
  'privacy',
  'high',
  'agenda',
  'Em',
  'City',
  'home',
  'many',
  'Aryans',
  'Muslims',
  'gangstas',
  'Latinos',
  'Christians',
  'Italians',
  'Irish',
  'scuffles',
  'death',
  'stares',
  'dod

In [ ]:
model =Word2Vec(sentences=words_in_sentences,
                size=200)

In [ ]:
model.wv.most_similar('film', topn=10)

[('movie', 0.7739988565444946),
 ('documentary', 0.5706967115402222),
 ('picture', 0.5681610107421875),
 ('flick', 0.5625534653663635),
 ('films', 0.5460941195487976),
 ('cinema', 0.497608482837677),
 ('product', 0.4884790778160095),
 ('thriller', 0.4598701596260071),
 ('masterpiece', 0.4533488154411316),
 ('effort', 0.4530629515647888)]

In [ ]:
model.wv.similar_by_word('film')

[('movie', 0.7739988565444946),
 ('documentary', 0.5706967115402222),
 ('picture', 0.5681610107421875),
 ('flick', 0.5625534653663635),
 ('films', 0.5460941195487976),
 ('cinema', 0.497608482837677),
 ('product', 0.4884790778160095),
 ('thriller', 0.4598701596260071),
 ('masterpiece', 0.4533488154411316),
 ('effort', 0.4530629515647888)]

In [ ]:
model.wv.get_vector('film')

array([ 0.75831085,  1.7398471 ,  0.03210738,  0.657718  , -1.353816  ,
        0.22059982, -0.30105463, -0.87791914, -0.76937896,  0.46714416,
        1.4324867 , -0.02015179, -1.6338803 , -2.1285787 , -0.08233839,
       -0.09128975,  0.6493269 ,  1.2729479 ,  1.385679  ,  0.5449528 ,
       -0.06368878,  0.99287933, -1.1569724 , -0.69624364, -1.6752893 ,
        0.7623769 , -0.56725454,  2.1218174 ,  0.6784344 ,  0.06385574,
        0.23399767,  0.5424006 , -0.22409043, -0.3822369 , -0.22386734,
        0.15147485,  0.5364539 , -0.25013238,  1.1204374 ,  0.78397065,
        0.3536187 ,  0.4582363 , -0.9873071 ,  0.36627856,  0.3092044 ,
        0.5097918 ,  1.8227621 ,  0.5991831 ,  0.84386504,  0.28898644,
        0.41134164, -0.6339339 ,  1.2547752 , -1.7232333 , -0.36762792,
        0.7213792 ,  0.88445556, -0.5936839 , -0.48862818, -0.54580307,
        0.21897164,  1.4736363 , -0.45235384, -1.2410722 , -0.49792457,
       -0.552732  ,  0.40249687,  0.7689655 , -0.86970603,  0.03

In [ ]:
model.wv.distance('boy','girl')

0.14262962341308594

In [ ]:
model.wv.n_similarity(['boy','male'],['girl','female'])

0.9042905

In [ ]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

One
reviewers
mentioned
watching
Oz
episode
hooked
They
right
exactly


In [ ]:
voc = list(model.wv.vocab)
voc

['One',
 'reviewers',
 'mentioned',
 'watching',
 'Oz',
 'episode',
 'hooked',
 'They',
 'right',
 'exactly',
 'happened',
 'br',
 'The',
 'first',
 'thing',
 'struck',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'word',
 'GO',
 'Trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'This',
 'pulls',
 'punches',
 'regards',
 'drugs',
 'sex',
 'Its',
 'hardcore',
 'classic',
 'use',
 'It',
 'called',
 'OZ',
 'nickname',
 'given',
 'Oswald',
 'Maximum',
 'Security',
 'State',
 'focuses',
 'mainly',
 'Emerald',
 'City',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'privacy',
 'high',
 'agenda',
 'Em',
 'home',
 'many',
 'Aryans',
 'Muslims',
 'Latinos',
 'Christians',
 'Italians',
 'Irish',
 'death',
 'stares',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'away',
 'I',
 'would',
 'say',
 'main',
 'appeal',
 'due',
 'fact',
 'goes',
 'shows',
 'dare',
 'Forget',
 'pretty',
 'pictures',
 'painted',
 'mainstream',
 'aud

In [ ]:
def avg_w2vec(sentences):
    transformed=[]
    for sentence in sentences:
        count=0
        vector=np.zeros(200)
        for word in sentence.split():
            if word in voc:
                vector+=model.wv.get_vector(word)
                count+=1
        if count!=0:
            vector/=count
            transformed.append(vector)
    return np.array(transformed)

In [ ]:
x_train=avg_w2vec(X_train)
x_test=avg_w2vec(X_test)

In [ ]:
x_train.shape

(500, 200)

In [ ]:
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(x_train, y_train)

In [ ]:
clf = LogisticRegression()

In [ ]:
clf.fit(X_train, y_train)

LogisticRegression()

In [ ]:
kf = KFold(n_splits=10, random_state = 42, shuffle = True)
scores = cross_val_score(clf, X_train, y_train, cv = kf)

In [ ]:
print('Cross-validation scores:', scores)
print('Cross-validation accuracy: {:.4f} (+/- {:.4f})'.format(scores.mean(), scores.std() * 2))

Cross-validation scores: [0.84313725 0.82352941 0.94117647 0.82352941 0.88235294 0.8627451
 0.74       0.8        0.88       0.74      ]
Cross-validation accuracy: 0.8336 (+/- 0.1198)


In [ ]:
predictions = clf.predict(x_test)

validation = dict()

validation['accuracy'] = accuracy_score(y_test, predictions)
validation['precision'] = precision_score(y_test, predictions, average='macro')
validation['recall'] = recall_score(y_test, predictions, average='macro')
validation['f1'] = f1_score(y_test, predictions, average='macro')

In [ ]:
print('Validation results:\n', '-' * 12)
for v in validation:
    print('{}: {:.5f}'.format(v.title(), validation[v]))

Validation results:
 ------------
Accuracy: 0.82200
Precision: 0.82287
Recall: 0.82200
F1: 0.82188
